In [1]:
# st_dataframe -> stationary values
# bm_dataframe -> body massage values
# hos_dataframe -> hospital values

In [2]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

In [3]:
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [5]:
address = 'Kolkata'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/usr/lib/python3.7/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


22.5677459 88.3476023


## Stationary shop

In [6]:
search_query = 'Stationary Shop'
radius = 5000
print(search_query + ' .... OK!')

Stationary Shop .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=QGITIC5BRJSPQFIEDOINIWZUBWBUX3JUGOAJO2H1KC1HPT1T&client_secret=2FJXKPKYOMK4DISKLHL4DI111FPZYCTZIZW2IF55VCLK0WJF&ll=22.5677459,88.3476023&v=20180604&query=Stationary Shop&radius=5000&limit=30'

In [8]:
results = requests.get(url).json()
#results

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d106951735', 'name': 'M...",False,4fa66144e4b0c1088efdf00b,"7/1A, Lindsay St",IN,Kolkata,India,NaN,949,"[7/1A, Lindsay St, Kolkata 700087, West Bengal...","[{'label': 'display', 'lat': 22.55980572172046...",22.559806,88.350978,700087,West Bengal,The Raymond Shop,v-1537511227
1,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",False,4e2d948cbd414272e5e51394,"6, Kiran Shankar Roy Rd",IN,Kolkata,India,NaN,429,"[6, Kiran Shankar Roy Rd, Kolkata 700001, West...","[{'label': 'display', 'lat': 22.56973304112606...",22.569733,88.344026,700001,West Bengal,Mio Amore,v-1537511227
2,"[{'id': '4bf58dd8d48988d106951735', 'name': 'M...",False,50dadf2ce4b02f73b466fc17,"18, Poddar Court",IN,Kolkata,India,NaN,969,"[18, Poddar Court, Kolkata 700001, West Bengal...","[{'label': 'display', 'lat': 22.5735686585696,...",22.573569,88.354613,700001,West Bengal,The Raymond Shop,v-1537511227
3,"[{'id': '4bf58dd8d48988d119951735', 'name': 'W...",False,5730675e498e181fdcb17e7b,"201/202, Chandni Chowk",IN,Kolkata,India,NaN,770,"[201/202, Chandni Chowk, Kolkata 700072, West ...","[{'label': 'display', 'lat': 22.56519549279807...",22.565195,88.354572,700072,West Bengal,R. R. Shaw (Wine) F. L. Off Shop,v-1537511227
4,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",False,50890a69e4b0a7db99707136,"7, Sudder St",IN,Kolkata,India,NaN,1186,"[7, Sudder St, Kolkata 700016, West Bengal, In...","[{'label': 'display', 'lat': 22.55937275304981...",22.559373,88.354751,700016,West Bengal,Sunshine Shop,v-1537511227


In [10]:
dataframe.shape

(30, 17)

In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

# copying it to stationary shop dataframe

st_dataframe = dataframe_filtered.copy()
st_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Raymond Shop,Men's Store,"7/1A, Lindsay St",IN,Kolkata,India,NaN,949,"[7/1A, Lindsay St, Kolkata 700087, West Bengal...","[{'label': 'display', 'lat': 22.55980572172046...",22.559806,88.350978,700087,West Bengal,4fa66144e4b0c1088efdf00b
1,Mio Amore,Bakery,"6, Kiran Shankar Roy Rd",IN,Kolkata,India,NaN,429,"[6, Kiran Shankar Roy Rd, Kolkata 700001, West...","[{'label': 'display', 'lat': 22.56973304112606...",22.569733,88.344026,700001,West Bengal,4e2d948cbd414272e5e51394
2,The Raymond Shop,Men's Store,"18, Poddar Court",IN,Kolkata,India,NaN,969,"[18, Poddar Court, Kolkata 700001, West Bengal...","[{'label': 'display', 'lat': 22.5735686585696,...",22.573569,88.354613,700001,West Bengal,50dadf2ce4b02f73b466fc17
3,R. R. Shaw (Wine) F. L. Off Shop,Wine Shop,"201/202, Chandni Chowk",IN,Kolkata,India,NaN,770,"[201/202, Chandni Chowk, Kolkata 700072, West ...","[{'label': 'display', 'lat': 22.56519549279807...",22.565195,88.354572,700072,West Bengal,5730675e498e181fdcb17e7b
4,Sunshine Shop,Clothing Store,"7, Sudder St",IN,Kolkata,India,NaN,1186,"[7, Sudder St, Kolkata 700016, West Bengal, In...","[{'label': 'display', 'lat': 22.55937275304981...",22.559373,88.354751,700016,West Bengal,50890a69e4b0a7db99707136


In [12]:
st_dataframe.name

0                                      The Raymond Shop
1                                             Mio Amore
2                                      The Raymond Shop
3                      R. R. Shaw (Wine) F. L. Off Shop
4                                         Sunshine Shop
5                                       Corona Pet Shop
6                                           little shop
7                          Bazlul Karim Lamp Shade Shop
8                                         The Roll Shop
9                                             Mega Shop
10                                          Sajjad Shop
11                                    Adam's Kabab Shop
12                                     Family Book Shop
13                   Mafatlal Family Shop (New Silsons)
14                        Victor Brother's Antique Shop
15                                   Standard Meat Shop
16    SquareKey Pop Up Shop @Le Souk - Shisha (Dec. 12)
17                                      Sankar T

In [36]:
# create map of Toronto using latitude and longitude values
map_kol_st = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, cat, postcode in zip(st_dataframe['lat'], st_dataframe['lng'],  st_dataframe['categories'],  st_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#ef00eb',
        fill=True,
        fill_color='#f4b804',
        fill_opacity=0.7).add_to(map_kol_st)  
    
map_kol_st

## Body Massage Shop

In [13]:
search_query = 'Body Massage Shop'
radius = 5000
print(search_query + ' .... OK!')

Body Massage Shop .... OK!


In [14]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=QGITIC5BRJSPQFIEDOINIWZUBWBUX3JUGOAJO2H1KC1HPT1T&client_secret=2FJXKPKYOMK4DISKLHL4DI111FPZYCTZIZW2IF55VCLK0WJF&ll=22.5677459,88.3476023&v=20180604&query=Body Massage Shop&radius=5000&limit=30'

In [15]:
results = requests.get(url).json()
#results

In [16]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '52f2ab2ebcbc57f1066b8b3c', 'name': 'M...",False,54ae4877498e66887278dfe4,Acharya Jagadish Chandra Bose Rd,IN,Kolkata,India,2875,"[Acharya Jagadish Chandra Bose Rd, Kolkata 700...","[{'label': 'display', 'lat': 22.54270369356614...",22.542704,88.354475,700017,West Bengal,Body Massage Parlor,v-1537511241
1,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",False,4eba49a64fc6624001a6a535,Forum Mall,IN,Kolkata,India,3312,"[Forum Mall, Kolkata 700020, West Bengal, India]","[{'label': 'display', 'lat': 22.53818507493977...",22.538185,88.351289,700020,West Bengal,The Body Shop,v-1537511241
2,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",False,52c0e16f498e1b8228e02e8a,Quest Mall,IN,Kolkata,India,3656,"[Quest Mall, Kolkata 700017, West Bengal, India]","[{'label': 'display', 'lat': 22.53939401809910...",22.539394,88.365558,700017,West Bengal,The Body Shop,v-1537511241
3,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",False,5288e7cb11d2ef11ed13dac2,NaN,IN,Kolkata,India,4476,"[Kolkata, West Bengal, India]","[{'label': 'display', 'lat': 22.56963043138064...",22.569630,88.391100,NaN,West Bengal,The Body Shop,v-1537511241


In [17]:
dataframe.shape

(4, 16)

In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered.head()
bm_dataframe = dataframe_filtered.copy()
bm_dataframe.head()

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Body Massage Parlor,Massage Studio,Acharya Jagadish Chandra Bose Rd,IN,Kolkata,India,2875,"[Acharya Jagadish Chandra Bose Rd, Kolkata 700...","[{'label': 'display', 'lat': 22.54270369356614...",22.542704,88.354475,700017,West Bengal,54ae4877498e66887278dfe4
1,The Body Shop,Cosmetics Shop,Forum Mall,IN,Kolkata,India,3312,"[Forum Mall, Kolkata 700020, West Bengal, India]","[{'label': 'display', 'lat': 22.53818507493977...",22.538185,88.351289,700020,West Bengal,4eba49a64fc6624001a6a535
2,The Body Shop,Cosmetics Shop,Quest Mall,IN,Kolkata,India,3656,"[Quest Mall, Kolkata 700017, West Bengal, India]","[{'label': 'display', 'lat': 22.53939401809910...",22.539394,88.365558,700017,West Bengal,52c0e16f498e1b8228e02e8a
3,The Body Shop,Cosmetics Shop,NaN,IN,Kolkata,India,4476,"[Kolkata, West Bengal, India]","[{'label': 'display', 'lat': 22.56963043138064...",22.569630,88.391100,NaN,West Bengal,5288e7cb11d2ef11ed13dac2


In [32]:
# create map of Toronto using latitude and longitude values
map_kol_bm = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, cat, postcode in zip(bm_dataframe['lat'], bm_dataframe['lng'],  bm_dataframe['categories'],  bm_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kol_bm)  
    
map_kol_bm

In [19]:
bm_dataframe.name

0    Body Massage Parlor
1          The Body Shop
2          The Body Shop
3          The Body Shop
Name: name, dtype: object

In [20]:
search_query = 'Hospital'
radius = 5000
print(search_query + ' .... OK!')

Hospital .... OK!


In [21]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=QGITIC5BRJSPQFIEDOINIWZUBWBUX3JUGOAJO2H1KC1HPT1T&client_secret=2FJXKPKYOMK4DISKLHL4DI111FPZYCTZIZW2IF55VCLK0WJF&ll=22.5677459,88.3476023&v=20180604&query=Hospital&radius=5000&limit=30'

In [22]:
results = requests.get(url).json()
#results

In [23]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1b3941735', 'name': 'M...",False,4e71c94cd4c0f5c149678109,"88, College St",IN,Kolkata,India,NaN,1354,"[88, College St, Kolkata 700073, West Bengal, ...","[{'label': 'display', 'lat': 22.57160065568263...",22.571601,88.360099,700073,West Bengal,Medical College and Hospital,v-1537511254
1,"[{'id': '4d4b7105d754a06372d81259', 'name': 'C...",False,4e16b775d4c0c7a8fbb4b1c6,"114, Acharya Jagadish Chandra Bose Rd",IN,Kolkata,India,NaN,2897,"[114, Acharya Jagadish Chandra Bose Rd, Kolkat...","[{'label': 'display', 'lat': 22.55075471877546...",22.550755,88.368959,700014,West Bengal,Dr. R. Ahmed Dental College and Hospital,v-1537511254
2,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4e04abee81dc9d212d106c85,"1, Lala Lajpat Rai Sarani",IN,Kolkata,India,NaN,3243,"[1, Lala Lajpat Rai Sarani, Kolkata 700020, We...","[{'label': 'display', 'lat': 22.53861117906464...",22.538611,88.348135,700020,West Bengal,Sambhunath Pandit Hospital,v-1537511254
3,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,55946eec498ef7546583e73c,"120/1, Andul Road, Near Nabanna, Howrah",IN,Kolkata,India,NaN,3157,"[120/1, Andul Road, Near Nabanna, Howrah, Kol...","[{'label': 'display', 'lat': 22.56364992, 'lng...",22.563650,88.317204,711103,West Bengal,Narayana Superspeciality Hospital,v-1537511254
4,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4feefe20ebca87c84f47bb45,Acharya Prafulla Chandra Rd,IN,Kolkata,India,NaN,3170,"[Acharya Prafulla Chandra Rd, Kolkata 700006, ...","[{'label': 'display', 'lat': 22.58179941882081...",22.581799,88.374431,700006,West Bengal,The Calcutta Homoeopathic Medical College and ...,v-1537511254


In [24]:
dataframe.shape

(30, 17)

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered.head()
hos_dataframe = dataframe_filtered.copy()
hos_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Medical College and Hospital,Medical School,"88, College St",IN,Kolkata,India,NaN,1354,"[88, College St, Kolkata 700073, West Bengal, ...","[{'label': 'display', 'lat': 22.57160065568263...",22.571601,88.360099,700073,West Bengal,4e71c94cd4c0f5c149678109
1,Dr. R. Ahmed Dental College and Hospital,College & University,"114, Acharya Jagadish Chandra Bose Rd",IN,Kolkata,India,NaN,2897,"[114, Acharya Jagadish Chandra Bose Rd, Kolkat...","[{'label': 'display', 'lat': 22.55075471877546...",22.550755,88.368959,700014,West Bengal,4e16b775d4c0c7a8fbb4b1c6
2,Sambhunath Pandit Hospital,Hospital,"1, Lala Lajpat Rai Sarani",IN,Kolkata,India,NaN,3243,"[1, Lala Lajpat Rai Sarani, Kolkata 700020, We...","[{'label': 'display', 'lat': 22.53861117906464...",22.538611,88.348135,700020,West Bengal,4e04abee81dc9d212d106c85
3,Narayana Superspeciality Hospital,Hospital,"120/1, Andul Road, Near Nabanna, Howrah",IN,Kolkata,India,NaN,3157,"[120/1, Andul Road, Near Nabanna, Howrah, Kol...","[{'label': 'display', 'lat': 22.56364992, 'lng...",22.563650,88.317204,711103,West Bengal,55946eec498ef7546583e73c
4,The Calcutta Homoeopathic Medical College and ...,Hospital,Acharya Prafulla Chandra Rd,IN,Kolkata,India,NaN,3170,"[Acharya Prafulla Chandra Rd, Kolkata 700006, ...","[{'label': 'display', 'lat': 22.58179941882081...",22.581799,88.374431,700006,West Bengal,4feefe20ebca87c84f47bb45


In [33]:
# create map of Toronto using latitude and longitude values
map_kol_hos = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, cat, postcode in zip(hos_dataframe['lat'], hos_dataframe['lng'],  hos_dataframe['categories'],  hos_dataframe['postalCode']):
    label = '{}, {}'.format(postcode, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kol_hos)  
    
map_kol_hos

In [26]:
hos_dataframe.shape

(30, 15)

In [27]:
hos_dataframe.name

0                          Medical College and Hospital
1              Dr. R. Ahmed Dental College and Hospital
2                            Sambhunath Pandit Hospital
3                     Narayana Superspeciality Hospital
4     The Calcutta Homoeopathic Medical College and ...
5         Nil Ratan Sarkar Medical College and Hospital
6                                  SVS Marwari Hospital
7                  Achintya Mohan Homoeopathic Hospital
8                                    Sree jain hospital
9                    Chittaranjan Shishu Sadan Hospital
10                                     Islamia hospital
11               R. G. Kar Medical College and Hospital
12             Calcutta Lions Bimal Poddar Eye Hospital
13                                        GDDI Hospital
14                               Lady Dufferin Hospital
15                                     B.R.S.H Hospital
16                         B. R. Singh Railway Hospital
17                                 Nightingale H